# Context Managers

## The `with` statement

The `with` statement simplifies exception handling by encapsulating common preparation and cleanup tasks in so-called context managers. The `with` statement is used to wrap the execution of a block with methods defined by a context manager. Context managers are defined by implementing two special methods: `__enter__()` and `__exit__()`.

In [ ]:
with open("file_name.txt", "w") as file:
   file.write("How you gonna win when you ain't right within?")

## Class Based Context Managers

A class based context manager is a class that implements `__enter__()` and `__exit__()` methods. The `__enter__()` method is called when the `with` block is entered, and the `__exit__()` method is called when the `with` block is exited.

In [1]:
class ContextManager:
  def __init__(self):
    print('Initializing class...')
 
  def __enter__(self):
    print('Entering context...')
 
  def __exit__(self, *exc):
    print('Exiting context...')

with ContextManager() as cm:
  print('Code inside with statement')

Initializing class...
Entering context...
Code inside with statement
Exiting context...


## Class Based Context Manager II

In [ ]:
class WorkWithFile:
  def __init__(self, file, mode):
    self.file = file
    self.mode = mode
 
  def __enter__(self):
    self.opened_file = open(self.file, self.mode)
    return self.opened_file
 
  def __exit__(self, *exc):
    self.opened_file.close()

with WorkWithFile("file.txt", "r") as file:
  print(file.read())

## Handling Exceptions I

When an exception is thrown in the `with` block, the `__exit__()` method is called with three arguments: the exception type, the exception value, and the traceback. If the `__exit__()` method returns `True`, the exception is suppressed. If it returns `False`, the exception is re-raised.

In [2]:
class OpenFile:
 
 def __init__(self, file, mode):
   self.file = file
   self.mode = mode

 def __enter__(self):
   self.opened_file = open(self.file, self.mode)
   return self.opened_file
 
 def __exit__(self, exc_type, exc_val, traceback):
   print(exc_type)
   print(exc_val)
   print(traceback)
   self.opened_file.close()

with OpenFile("file.txt", "r") as file:
  # .see() is not a real method
  print(file.see())

<class 'AttributeError'>
'_io.TextIOWrapper' object has no attribute 'see'


AttributeError: '_io.TextIOWrapper' object has no attribute 'see'

## Handling Exceptions II

An exception that occurs in a context manager can be handled in two ways:

- if we want to throw the exception, we can return `False` from the `__exit__()` method.
- if we want to suppress the exception, we can return `True` from the `__exit__()` method.

In [9]:
class OpenFile:
 
  def __init__(self, file, mode):
    self.file = file
    self.mode = mode
 
  def __enter__(self):
    self.opened_file = open(self.file, self.mode)
    return self.opened_file
 
  def __exit__(self, exc_type, exc_val, traceback):
    print(exc_type, exc_val, traceback)
    print("The exception has been handled")
    self.opened_file.close()
    return True


In [10]:
with OpenFile("file.txt", "r") as file:
  # .see() is not a real method
  print(file.see())

with OpenFile("file.txt", "r") as file:
    print(file.read())

<class 'AttributeError'> '_io.TextIOWrapper' object has no attribute 'see' <traceback object at 0x10b94ab00>
The exception has been handled
foobar
None None None
The exception has been handled


Since the `__exit__()` returns `True` in the example, the exception is suppressed. The second `with` statement and `print()` statement is executed, and the program continues.

Additionally, we can choose to handle the exception in the `__exit__()` method.

In [ ]:
class OpenFile:
 
  def __init__(self, file, mode):
    self.file = file
    self.mode = mode
 
  def __enter__(self):
    self.opened_file = open(self.file, self.mode)
    return self.opened_file
 
  def __exit__(self, exc_type, exc_val, traceback):
    self.file.close()
    if isinstance(exc_val, TypeError):
      # Handle TypeError here...
      print("The exception has been handled")
      return True

An `if` statement is used to check if an exception occurred. If an exception occurred, the exception is printed, and the method returns `True`.

## Introduction to Contextlib

The `contextlib` module provides utilities for common tasks involving the `with` statement. The `contextlib` module includes a decorator and context manager that can be used to create simple context managers.

Documentation: https://docs.python.org/3/library/contextlib.html

In [1]:
from contextlib import contextmanager

@contextmanager
def open_file_contextlib(file, mode):
    opened_file = open(file, mode)
    try:
        yield opened_file
    finally:
        opened_file.close()

```python
@contextmanager
def generator_function(<parameters>):
    <setup section - equivalent to __enter__ >
    try:
        yield <value>
    finally:
        <cleanup section - equivalent to __exit__ >
```

In [2]:
with open_file_contextlib('file.txt', 'w') as opened_file:
    opened_file.write('We just made a context manager using contexlib')

## Contextlib Error Handling

There are two ways to handle exceptions in the `contextlib` module:
- do nothing by excluding the `except` block
- handle the exception by including the `except` block

In [3]:
from contextlib import contextmanager

@contextmanager
def open_file_contextlib(file, mode):
    open_file = open(file, mode)
 
    try:
        yield open_file
    # Exception Handling
    except Exception as exception:
        print('We hit an error: ' + str(exception))
    finally:
        open_file.close()
 
with open_file_contextlib('file.txt', 'w') as opened_file:
    opened_file.sign('We just made a context manager using contexlib')

We hit an error: '_io.TextIOWrapper' object has no attribute 'sign'


## Nested Context Managers

- work with information from multiple files
- copy the same information to multiple files
- copy information from one file to another

In this example, we open two files, `teacher.txt` and `student.txt`, and copy the contents of `student.txt` to `teacher.txt`.

In [ ]:
with open('teacher.txt', 'w') as teacher, open('student.txt', 'r') as student:
 teacher.write(student.read())

- The `with` statement is being called once but is being used to open two files.
- Each file is opened in a separate context manager.
- Each file has different modes.

We can also write the code using nested `with` statements.

In [ ]:
with open("teacher.txt", "w") as teacher:
   with open("student.txt", "r") as student:
     teacher.write(student.read())

- The `with` statement is called twice.
- Though slightly longer, this method is more readable.